## **RAG의 핵심, 문서 검색기 Retriever**
### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**
**Chroma DB에 문서 벡터 저장**

In [ ]:
import os
from dotenv import load_dotenv

# 환경변수 읽어오기
load_dotenv(override=True)  # .env 파일을 덮어쓰기 모드로 읽기

# 환경변수 불러오기
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

print(f"openai key values ::: {openai_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"anthropic key values ::: {anthropic_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"huggingface_token::: {huggingface_token}")  # 테스트용 (실제 서비스에서는 print 금지)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
result = embedding_model.embed_query("테스트 문장입니다.")
print(result[:5])  #

Test 소스

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

#헌법 PDF 파일 로드

# PDF파일 불러올 객체 PyPDFLoader 선언 - window11 
loader = PyPDFLoader(r"G:\내 드라이브\LLM-RAG-LangChain\대한민국헌법(헌법제1호).pdf")

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
# loader = PyPDFLoader(
#     r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
# )
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# docs = text_splitter.split_documents(pages)

# Chroma를 제거하고 임베딩만 추출
docs = text_splitter.split_documents(pages[:3])  # 소량만 사용

# 임베딩 결과 확인
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
for doc in docs:
    vec = embedding_model.embed_query(doc.page_content)
    print(vec[:5])

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings                           
from langchain_community.vectorstores import Chroma
# from langchain_chroma import Chroma

import shutil

import logging

# 기본 로깅 설정
# logging.basicConfig(level=logging.DEBUG)

# ChromaDB 관련 로거 활성화
# logging.getLogger("chromadb").setLevel(logging.DEBUG)
# logging.getLogger("chromadb.db").setLevel(logging.DEBUG)
# logging.getLogger("chromadb.telemetry").setLevel(logging.INFO)

#헌법 PDF 파일 로드

# PDF파일 불러올 객체 PyPDFLoader 선언 - window11 
# loader = PyPDFLoader(r"G:\내 드라이브\LLM-RAG-LangChain\대한민국헌법(헌법제1호).pdf")

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages[:5])

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
# db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))
# db = Chroma.from_documents(
#     docs,
#     OpenAIEmbeddings(model="text-embedding-3-small"),
#     persist_directory="./chroma_db"  # 폴더 직접 지정
# )

# shutil.rmtree("C:/temp/chroma_db", ignore_errors=True)

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

# shutil.rmtree("./chroma_db", ignore_errors=True)
db = Chroma.from_documents(docs, embedding_model, persist_directory="./chroma_db")

# db = Chroma(
#     embedding_function=embedding_model,
#     persist_directory="C:/temp/chroma_db",
#     collection_name="my_collection"
# )

# # 소규모 배치로 나누어 삽입
# batch_size = 5
# for i in range(0, len(docs), batch_size):
#     batch = docs[i:i + batch_size]
#     db.add_documents(batch)

**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [ ]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

In [ ]:
unique_docs

### **<span style="color:yellow">재정렬 기법(Reorder or Rerank)</span>**

**[Long-Context Reorder 없이 유사 문서 출력]**

In [ ]:
#Chroma dimension 관련 에러 발생 시 실행
# Chroma().delete_collection()

# pip install faiss-gpu

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAI

from langchain_openai import OpenAIEmbeddings

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = FAISS.from_texts(texts, OpenAIEmbeddings(model = 'text-embedding-3-small')).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.invoke(query)
for i in docs:
  print(i.page_content)

**[Long-Context Reorder 활용하여 유사 문서 출력]**

In [ ]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
#LongContextReorder 선언
reordering = LongContextReorder()
#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
for i in reordered_docs:
  print(i.page_content)

**필요없는 문서는 삭제, Contextual Compression, 맥락 압축 기법(Context Compression)**

In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)
db = FAISS.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))
retriever =db.as_retriever()

docs = retriever.invoke("대통령의 임기는?") 
pretty_print_docs(docs)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model ='gpt-4o-mini', temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "대통령의 임기는?"
)
pretty_print_docs(compressed_docs)

### **<span style="color:yellow">가상 문서로 유사 문서 탐색, HyDE(Hypothetical Document Embedding)</span>**

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """
당신은 LangChain, LangGraph, LangServe, LangSmith라는 LLM 기반 애플리케이션을 구축하기 위한 일련의 소프트웨어에 대한 전문가입니다.

LangChain은 LLM 애플리케이션을 구축하기 위해 쉽게 구성할 수 있는 대규모 통합 세트를 제공하는 Python 프레임워크입니다.
LangGraph는 상태 저장, 멀티 액터 LLM 애플리케이션을 쉽게 구축할 수 있는 LangChain 위에 구축된 Python 패키지입니다.
LangServe는 REST API로 LangChain 애플리케이션을 쉽게 배포할 수 있는 LangChain 위에 구축된 Python 패키지입니다.
LangSmith는 LLM 애플리케이션 추적 및 테스트를 쉽게 할 수 있는 플랫폼입니다.

사용자 질문에 최선을 다해 답변하세요. 사용자 질문에 대한 튜토리얼을 작성하는 것처럼 답변하세요.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
qa_no_context = prompt | llm | StrOutputParser()

In [18]:
answer = qa_no_context.invoke(
    {
        "question": "Chain을 구축할 때 멀티모달 모델을 활용하는 방법과 Chain을 REST API로 전환하는 방법"
    }
)
print(answer)

Chain을 구축할 때 멀티모달 모델을 활용하고 이를 REST API로 전환하는 방법에 대해 단계별로 설명하겠습니다. 이 튜토리얼에서는 LangChain과 LangServe를 사용하여 멀티모달 Chain을 구축하고 이를 REST API로 배포하는 과정을 다룹니다.

### 1단계: 멀티모달 모델 준비하기

멀티모달 모델은 텍스트, 이미지, 오디오 등 다양한 형태의 데이터를 처리할 수 있는 모델입니다. 예를 들어, OpenAI의 CLIP 모델은 텍스트와 이미지를 함께 처리할 수 있습니다. 먼저 필요한 라이브러리를 설치합니다.

```bash
pip install langchain openai
```

### 2단계: 멀티모달 Chain 구축하기

이제 LangChain을 사용하여 멀티모달 Chain을 구축해 보겠습니다. 아래는 텍스트와 이미지를 입력으로 받아 처리하는 간단한 Chain의 예입니다.

```python
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# 텍스트와 이미지를 처리하는 프롬프트 템플릿 생성
prompt_template = PromptTemplate(
    input_variables=["text", "image"],
    template="Given the text: {text} and the image: {image}, provide a detailed analysis."
)

# LLM 초기화
llm = OpenAI(model="gpt-4")

# Chain 생성
chain = LLMChain(llm=llm, prompt=prompt_template)

# Chain 테스트
response = chain.run(text="A beautif

In [19]:
from langchain_core.runnables import RunnablePassthrough

hyde_chain = RunnablePassthrough.assign(hypothetical_document=qa_no_context)

hyde_chain.invoke(
    {
        "question": "Chain을 구축할 때 멀티모달 모델을 활용하는 방법과 Chain을 REST API로 전환하는 방법"
    }
)

{'question': 'Chain을 구축할 때 멀티모달 모델을 활용하는 방법과 Chain을 REST API로 전환하는 방법',
 'hypothetical_document': 'Chain을 구축할 때 멀티모달 모델을 활용하고, 이를 REST API로 전환하는 방법에 대해 단계별로 설명하겠습니다. 이 튜토리얼에서는 LangChain과 LangServe를 사용하여 멀티모달 Chain을 구축하고 이를 REST API로 배포하는 과정을 다룹니다.\n\n### 1단계: 멀티모달 모델 준비하기\n\n멀티모달 모델은 텍스트, 이미지, 오디오 등 다양한 형태의 데이터를 처리할 수 있는 모델입니다. 예를 들어, OpenAI의 CLIP 모델은 텍스트와 이미지를 함께 처리할 수 있습니다. 먼저 필요한 라이브러리를 설치합니다.\n\n```bash\npip install langchain\npip install torch torchvision\n```\n\n### 2단계: 멀티모달 Chain 구축하기\n\n이제 LangChain을 사용하여 멀티모달 Chain을 구축해 보겠습니다. 아래는 텍스트와 이미지를 입력으로 받아 처리하는 간단한 Chain의 예입니다.\n\n```python\nfrom langchain import Chain\nfrom langchain.llms import OpenAI\nfrom langchain.prompts import PromptTemplate\nfrom langchain.chains import SimpleSequentialChain\n\n# 텍스트와 이미지를 처리할 LLM 초기화\nllm = OpenAI(model="gpt-3.5-turbo")\n\n# 텍스트 프롬프트 템플릿 생성\ntext_prompt = PromptTemplate(\n    input_variables=["text"],\n    template="이 텍스트에 대한 설명을 제공하세요: {text}"\n)\n\n# 이미지 프롬프트 템플릿 생성\nimage_prompt = PromptT